# Read raw text corpora

In [1]:
import os
import chardet
import magic 
import pandas as pd
from typing import Tuple, List, Iterable

def read_raw_corpus(files: Iterable[str], corpus: str) -> Tuple[pd.DataFrame, List[str]]:
    """
    Reads all files from file and creates a dataframe containing:
        * Raw text
        * title
        * year of publication
        * period label
        * genre label
        * region label
        * corpus label (has to set as param)
    
    NOTE: The Corpus label must be either 'innsbruck' or 'manchester'.
    If it is 'manchester' another split is used to read the metadata from the filenames.
    
    Also a list of tuples containing the files which could not be processed 
    and the error which was thrown while trying to do so is returned.
    
    :param file Liste of filepaths
    :param corpus corpus label string
    :return a dataframe + a list files which could not be processed
    """
    
    data = []
    errors = []
    for file in files:
    
        m_file = magic.Magic(mime_encoding=True)
        file_encoding = m_file.from_buffer(file)
        file = file.encode(file_encoding).decode('UTF-8')
        filename = os.path.basename(file)
        genre, period, region, year, title = filename.split('_')[:5]
        if corpus.lower() == 'manchester':
            if title.endswith('RAW.txt'):
                title = title.replace('RAW.txt', '')
            else:
                title = title.replace('.txt', '')

        with open(file, 'rb') as f:
            text_bytes = f.read()
            try:
                encoding_info = chardet.detect(text_bytes)
                text = text_bytes.decode(encoding_info['encoding'])
            except Exception as e:
                errors.append((file, str(e)))
                continue

        data.append({
            'filename': file,
            'text': text,
            'title': title,
            'genre': genre,
            'period': period,
            'region': region,
            'year': year,
            'filename': filename,
            'corpus': corpus
        })
        
    df = pd.DataFrame.from_dict(data)
    df = df.set_index('filename')
    return df, errors



# Read raw innsbruck corpus files into a dataframe

In [2]:
import glob

innsbruck_raw_files = glob.glob('GermInnC/**/*.txt')
print(len(innsbruck_raw_files))

638


In [3]:
innsbruck_raw_df, errors = read_raw_corpus(innsbruck_raw_files, 'innsbruck')
print(errors)
print(innsbruck_raw_df.shape)
innsbruck_raw_df.head()

[]
(638, 7)


,corpus,genre,period,region,text,title,year
filename,,,,,,,
NEWS_P6_OOD_1915_Innsbrucker Nachrichten_RAW.txt,innsbruck,NEWS,P6,OOD,Aus Stadt und Land.\n\n(Trauergottesdienst.) Z...,Innsbrucker Nachrichten,1915
NEWS_P6_OMD_1928_Arbeiterstimme_RAW.txt,innsbruck,NEWS,P6,OMD,Aus Dresdens Zigaretten Industrie\r\n\tDresden...,Arbeiterstimme,1928
HUMA_P6_WMD_1920_Evangelischer Sozialismus_RAW .txt,innsbruck,HUMA,P6,WMD,Einleitung.\r\n § 1.\r\n Glaub...,Evangelischer Sozialismus,1920
DRAM_P6_WMD_1915_Heimkehr_RAW.txt,innsbruck,DRAM,P6,WMD,Zimmer im Lenz'schen Hause.\r\n(Clara steht au...,Heimkehr,1915
NEWS_P6_OOD_1914_Wiener Zeitung_RAW.txt,innsbruck,NEWS,P6,OOD,Lokales.\n(Feuer im Allgemeinen Krankenhause.)...,Wiener Zeitung,1914


In [4]:
innsbruck_raw_df.to_csv('innsbruck_dataset.csv', encoding='UTF-8')

# Read raw manchester corpus files into a dataframe

In [5]:
manchester_raw_files = glob.glob('Manchester/RAW/*.txt')

In [6]:
manchester_raw_df, errors = read_raw_corpus(manchester_raw_files, 'manchester')
print(errors)
print(manchester_raw_df.shape)
manchester_raw_df.head()

[]
(336, 7)


,corpus,genre,period,region,text,title,year
filename,,,,,,,
HUMA_P3_WMD_1777_HomburgRAW.txt,manchester,HUMA,P3,WMD,Nachricht von den Alterthu&#868;mern in dem Ge...,Homburg,1777
NEWS_P3_NoD_1786_wolfenbuettel1.txt,manchester,NEWS,P3,NoD,"Zeitung\r\nfür\r\nStädte, Flecken und Dörfer,\...",wolfenbuettel1,1786
NARR_P1_NoD_1658_MorgenlaendischRAW.txt,manchester,NARR,P1,NoD,Das zwey vnd dreysigste Capitel.\n Des Pr...,Morgenlaendisch,1658
SCIE_P1_WMD_1680_EpidemicaRAW.txt,manchester,SCIE,P1,WMD,Das XX. Capitel.\n Von den Schnecken.\n ...,Epidemica,1680
NEWS_P2_WMD_1701_hanau2.txt,manchester,NEWS,P2,WMD,Extraordinari Europæische Zeitung. 1701. Num. ...,hanau2,1701


In [7]:
manchester_raw_df.to_csv('manchester_dataset.csv', encoding='UTF-8')

# Concat raw dataframes

In [8]:
full_raw_df = pd.concat((manchester_raw_df, innsbruck_raw_df))
print(full_raw_df.shape)
full_raw_df.head()

(974, 7)


,corpus,genre,period,region,text,title,year
filename,,,,,,,
HUMA_P3_WMD_1777_HomburgRAW.txt,manchester,HUMA,P3,WMD,Nachricht von den Alterthu&#868;mern in dem Ge...,Homburg,1777
NEWS_P3_NoD_1786_wolfenbuettel1.txt,manchester,NEWS,P3,NoD,"Zeitung\r\nfür\r\nStädte, Flecken und Dörfer,\...",wolfenbuettel1,1786
NARR_P1_NoD_1658_MorgenlaendischRAW.txt,manchester,NARR,P1,NoD,Das zwey vnd dreysigste Capitel.\n Des Pr...,Morgenlaendisch,1658
SCIE_P1_WMD_1680_EpidemicaRAW.txt,manchester,SCIE,P1,WMD,Das XX. Capitel.\n Von den Schnecken.\n ...,Epidemica,1680
NEWS_P2_WMD_1701_hanau2.txt,manchester,NEWS,P2,WMD,Extraordinari Europæische Zeitung. 1701. Num. ...,hanau2,1701


In [9]:
#full_raw_df.to_csv('full_dataset.csv', encoding='UTF-8')

# Read tagged corpora

In [10]:
import glob
tagged_files = glob.glob('GermInnC Tagged/**/*.txt')

In [11]:
import os
from typing import Iterable
tagged_data = []
for file in tagged_files:
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    tokens = []
    pos_tags = []
    lemmas = []
    with open(file, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.rstrip()
            try:
                token, pos_tag, lemma = line.split('\t')
            except ValueError:
                continue
            tokens.append(token)
            pos_tags.append(pos_tag)
            lemmas.append(lemma)

    tagged_data.append({
        'tokens': " ".join(tokens),
        'pos_tags': " ".join(pos_tags),
        'lemmas': " ".join(lemmas),
        'title': title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year,
        'corpus': 'innsbruck'
    })

In [12]:
tagged_data[0]

{'tokens': 'Lokales und Vermischtes . Der Kaiser in Breslau . Der Kaiser , der bekanntlich heute den Feierlichkeiten anläßlich der Vermählung der Gräfin Armgard zu Stolberg = Wernigerode mit dem Grafen Platen zu Hallermund in Landeshut beiwohnen wird , begibt sich gleich nach der Trauung nach Breslau , wo er heute Abend eintrifft . Der Kaiser wird sich , wie uns ein Privat = Telegramm aus Breslau meldet , sofort nach dem fürstbischhöflichen Palais begeben , um beim Kardinal Kopp das Souper einzunehmen . Die Rückreise des Kaisers nach Berlin erfolgt am Dienstag Nachmittag . – Die Kaiserin hat ihre Teilnahme an der Hochzeitsfeierlichkeit wegen leichter Unpäßlichkeit abgesagt . Gestern Vormittag wohnte der Kaiser dem Gottesdienst zu Potsdam bei und empfing um 12 ¾ Uhr den Staatsminister Budde . – Gestern Abend begab sich der Kaiser nach dem königlichen Schauspielhaus , um der Vorstellung von Blumenthals „ Mann wir altern “ und Rofstands „ Die Romantischen “ beizuwohnen . Im Schauspielhaus

In [13]:
import pandas as pd

df = pd.DataFrame.from_dict(tagged_data)
df.to_csv('tagged_dataset.csv', encoding='UTF-8')

# Tagged Manchester

In [14]:
tagged_files_2 = glob.glob('Manchester/LING-COL/*.txt')

In [15]:
import os
from typing import Iterable
tagged_data_2 = []
for file in tagged_files_2:
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    tokens = []
    norms = []
    pos_tags = []
    lemmas = []
    with open(file, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.rstrip()
            try:
                pID, token, norm, pos_tag, lemma, moph, p, rest = line.split('\t')
                #token, pos_tag, lemma = line.split('\t')
            except ValueError:
                continue
            
            tokens.append(token)
            norms.append(norm)
            pos_tags.append(pos_tag)
            lemmas.append(lemma)

    tagged_data_2.append({
        'tokens': " ".join(tokens),
        #'norm': " ".join(norms),
        'pos_tags': " ".join(pos_tags),
        'lemmas': " ".join(lemmas),
        'title': title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year,
        'corpus': 'manchester'
    })

In [16]:
tagged_data_2[0]

{'tokens': 'Aus dem Braunschweigischen . An die lieben Landleute . Meine lieben Freunde ! wir lesen manchmal die Zeitung , und es gefaͤllt uns vieles , was wir darin lesen . Man hoͤrt und sieht denn doch , wie es in der Welt hergeht , da giebt es Gutes und Boͤses durch einander . Dies Lesen und Wissen hat sein Angenehmes , auch sein Nuͤtzliches , und macht klug . Deswegen gefaͤllt mir unser Nachbar Heinrich , der lieset die Zeitungen , wo er ausspannt , und kann mitsprechen , und ist kein dummer Mann . Wie oft lernt man auch was aus den Zeitungen , das man wieder brauchen kann in der Haushaltung , beym Ackerbau , beym Viehe , in Kuͤche und Keller oder sonsten , davon man reinen Profit hat . Darum waͤre es immer gut , wenn wir die Zeitungen nur oͤftrer laͤsen . Aber das ist denn auch oft verzweifelt schwer . Die Zeitungen sind so groß , so weitlaͤuftig , oft ganze Bogen lang , da vergeht einem die Geduld , und unser einer hat mehr zu thun , als Zeitungen zu lesen : Und hat man denn erst

# Create the Manchester Tagged Dataframe and .csv

In [17]:
import pandas as pd
tagged_man_df = pd.DataFrame.from_dict(tagged_data_2)
tagged_man_df.shape

(336, 9)

In [18]:
tagged_man_df.to_csv('tagged_Mancheser_dataset.csv', encoding='UTF-8')

# Concat the tagged Datasets and save in .csv

In [19]:
full_tagged_df = pd.concat((tagged_man_df, df))
print(full_tagged_df.shape)
full_tagged_df

(974, 9)


,corpus,genre,lemmas,period,pos_tags,region,title,tokens,year
0,manchester,NEWS,aus d Braunschweigische . an d lieb Landleute ...,P3,APPR ART NA SENT APPR ART ADJA NN SENT PPOSAT ...,NoD,wolfenbuettel1.txt,Aus dem Braunschweigischen . An die lieben Lan...,1786
1,manchester,DRAM,"Ei was Schaath er d , wenn im grün Gruß unser ...",P2,"ITJ PWS NE PPER PDS $, KOUS APPRART ADJA NN PP...",WMD,Bookesbeutel.txt,"Ey was schadt ihm das , Wenn im gruͤnen Graß U...",1742
2,manchester,LEGA,von d Aiden d Cammer-Gerichts-Persohnen und Pa...,P2,"APPR ART NN ART NN KON NN $, ADV PAV PTKZU VVI...",WMD,GesetzBuch.txt,Von den Eyden der Cammer-Gerichts-Persohnen un...,1724
3,manchester,SERM,Abhandlung . d heutig Freytag sein derjenige W...,P2,"NN SENT ART ADJA NE VAFIN PDS NN $, APPR PRELS...",WMD,HeilBronnen.txt,Abhandlung . DEr heutige Freytag ist derjenige...,1721
4,manchester,NEWS,Madrid / d @ord@ September . an Don Manuel Ari...,P2,"NE $, ART CARD NN SENT APPR NE NE NE NN APPR N...",WMD,hanau2.txt,Madritt / den 8. Septembris . An Don Manuel Ar...,1701
5,manchester,SERM,Eingang . regieren jemand / so sein er sorgfäl...,P1,"NN SENT VVFIN PIS $, ADV VAFIN PPER ADJD SENT ...",OMD,SursumDeosum.txt,Eingang . REgieret iemand / so sei er sorgfaͤl...,1680
6,manchester,SCIE,d dritt Buch . Kapitel @ord@ was vor Person zu...,P1,ART ADJA NN SENT NN CARD PWS APPR NN APPRART N...,OMD,BergBau.txt,Das Dritte Buch . Cap. 1. Was vor Personen zum...,1700
7,manchester,SCIE,Abhandlung / von Schläussen und Roll Brücke . ...,P2,"NN $, APPR NN KON NN NN SENT ART CARD NN SENT ...",WOD,FangSchlaeussen.txt,Abhandlung / Von Schlaͤussen und Roll Bruͤcken...,1720
8,manchester,LEGA,lieb SECUNDUS . Titulus Primus . de TESTAMENTI...,P2,ADJA NE SENT NN NN SENT NE NE NE NE APPRART AD...,NoD,StadtRecht.txt,LIBER SECUNDUS . TITULUS PRIMUS . DE TESTAMENT...,1724
9,manchester,SERM,erst Teil . ein verwaysetes Land bleiben trost...,P3,ADJA NN SENT ART ADJA NN VVFIN ADJD APPR NE SE...,OMD,Trost.txt,Erster Theil . Ein verwaysetes Land bleibt tro...,1756


In [20]:
full_tagged_df.to_csv('full_taggeddataset.csv', encoding='UTF-8')

# Clean Datasets

In [21]:
import pandas as pd
import numpy as np
import re
#from sklearn.utils import shuffle

tagged_df = pd.read_csv('full_taggeddataset.csv')
full_raw_df = pd.read_csv('full_dataset.csv')

In [22]:
# Clean Dataset RAW
full_raw_df[full_raw_df.genre == "NEWS-P4"] = "NEWS"
full_raw_df.region = full_raw_df.region.str.upper()
full_raw_df = full_raw_df[full_raw_df.year != 'GesetzsammlungThÅringen']
full_raw_df = full_raw_df[full_raw_df.year != 'GesetzsammlungThüringen']
full_raw_df = full_raw_df[full_raw_df.year != '1851-54']
full_raw_df['text'] = full_raw_df['text'].str.replace('+',' ')
full_raw_df['text'] = full_raw_df['text'].str.replace('@ord@','')
#Regex
full_raw_df['text'] = full_raw_df['text'].str.replace(r'^d$','die')
full_raw_df['text'] = full_raw_df['text'].str.replace(r'\b\d+\w+\b','')
full_raw_df['text'] = full_raw_df['text'].str.replace(r'\b\w+\d+\b','')
full_raw_df['text'] = full_raw_df['text'].str.replace(r'\b\d+\b','')

full_raw_df['text'] = full_raw_df['text'].str.replace('seyn','sein')
full_raw_df['text'] = full_raw_df['text'].str.replace('er\|es\|sie','sich')
# Clean Dataset Tagged
tagged_df[tagged_df.genre == "NEWS-P4"] = "NEWS"
tagged_df.region = tagged_df.region.str.upper()
tagged_df = tagged_df[tagged_df.year != 'GesetzsammlungThÅringen']
tagged_df = tagged_df[tagged_df.year != 'GesetzsammlungThüringen']
tagged_df = tagged_df[tagged_df.year != '1851-54']
tagged_df['lemmas'] =tagged_df['lemmas'].str.replace('seyn','sein')
tagged_df['lemmas'] =tagged_df['lemmas'].str.replace('+',' ')
#Regex
tagged_df['lemmas'] =tagged_df['lemmas'].str.replace(r'\b\d+\w+\b','')
tagged_df['lemmas'] =tagged_df['lemmas'].str.replace(r'\b\w+\d+\b','')
tagged_df['lemmas'] =tagged_df['lemmas'].str.replace(r'\b\d\b','')
tagged_df['lemmas'] =tagged_df['lemmas'].str.replace(r'\bd\b','die')

tagged_df['lemmas'] =tagged_df['lemmas'].str.replace('@ord@','')
tagged_df['lemmas'] =tagged_df['lemmas'].str.replace('er\|es\|sie','sich')
#tagged_df = tagged_df[tagged_df.text.replace('seyn', 'sein')]
#tagged_df = tagged_df[tagged_df.genre != 'NEWS']
#Zahlen

In [23]:
full_raw_df.to_csv('full_dataset.csv', encoding='UTF-8')
tagged_df.to_csv('full_taggeddataset.csv', encoding='UTF-8')